<img src="./images/instagram_header.png" align="left" style="margin-bottom: 20px"/>

<h2> Web Appendix - Data Analysis </h2>

<p style="clear: both;">This online appendix complements the master thesis "Goodbye Likes, Hello Mental Health: How Hiding Like Counts Affects User Behavior & Self-Esteem":</p> 

<p><i>Likes are widely available on social network services and are known to influence people’s self-image. An emerging literature has started to look at potential detrimental effects of social media use among teenagers. We study how Instagram users’ posting frequency, variety, like behavior, and relative self-esteem are affected by an intervention in which like counts were hidden in selected treatment countries. Using a unique panel data set of individual users’ Instagram posts across multiple years, we find evidence that users posted more frequently and more varied than in the months prior to the intervention. On the other hand, the number of likes decreases as people are no longer influenced by others’ evaluations, especially among users with a small following. Further, in an experiment we show that the number of likes people see on others’ posts affects their relative self-esteem, and that users are more likely to self-disclose once they rate themselves more positively. These results are critical to understanding the dynamics on visual-based social media in order to foster a healthy online environment.</i></p>

<p>In this notebook, we perform the following steps (run this .ipynb-file locally for clickable anchors): </p>

F. [Outlier Screening](#outlier-screening)  
G. [Propensity Score Matching](#propensity-score-matching)  
H. [Differences in Differences](#differences-in-differences)  
I. [Randomized Experiment](#experiment)  

<i> Note: Steps A-E (data collection & preparation) can be found over <a href="https://github.com/RoyKlaasseBos/Hiding-Instagram-Likes/blob/master/Web_Appendix_Data_Collection_Preparation.ipynb">here</a>. </i>

In [1]:
# the line below may take a couple of minutes to run; in case you are unable to select a CRAN mirror in Jupyter Notebook: open RStudio and install the packages there which resolves the issue.
install.packages(c("RPostgreSQL", "erer", "ez", "MASS", "pscl", "psych", "plm", "reshape"))
library(RPostgreSQL) # dbGetQuery
library(erer) # ocME
library(ez) # ezANOVA
library(MASS) # polr
library(pscl) # pR2
library(psych) # alpha
library(plm) # plm
library(reshape) # melt

host = Sys.getenv(c("INSTAGRAM_DB_URL"))
password = Sys.getenv(c("INSTAGRAM_DB_KEY"))

drv = dbDriver("PostgreSQL")
con = dbConnect(drv, host=host, 
                port='5432', dbname='postgres',
                user='postgres', password=password)

Warning message:
“package ‘RPostgreSQL’ was built under R version 3.4.1”Loading required package: DBI
Loading required package: lmtest
Warning message:
“package ‘lmtest’ was built under R version 3.4.4”Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Warning message in as.POSIXlt.POSIXct(Sys.time()):
“unknown timezone 'zone/tz/2020a.1.0/zoneinfo/Europe/Amsterdam'”Classes and Methods for R developed in the
Political Science Computational Laboratory
Department of Political Science
Stanford University
Simon Jackman
hurdle and zeroinfl functions by Achim Zeileis
Warning message:
“package ‘reshape’ was built under R version 3.4.4”

<a id='outlier-screening'></a>
### F. Outlier Screening
Even though we use a stringent [Instagram consumers selection](#instagram-consumers-selection) procedure, it may sporadically occur that a user systematically differs in terms of the number of followers, followings, and the average number of likes of their image posts. To overcome this issue we use a multivariate outlier screening approach and remove these users from our sample before propensity score matching.

In [2]:
# split query into subqueries
query_1 = "SELECT x.username, CAST(followers_count AS NUMERIC), CAST(following_count AS NUMERIC), average_likes FROM" 
query_2 = "x INNER JOIN (SELECT username, AVG(total_likes) as average_likes FROM"
query_3 = "l GROUP BY username) l ON x.username = l.username"

query_user_data = function(user_profile, user){
    # paste queries and determine follower count, following count, and average number of likes by user
    query = paste(query_1, user_profile, query_2, user, query_3)
    return(dbGetQuery(con, query))
}

outlier_screening = function(df){
    # determine if the mahalanobis distance exceeds the threshold value
    mahal = mahalanobis(df[,-1], colMeans(df[,-1]), cov(df[,-1]), tol=1e-20)
    cutoff = qchisq(1-0.001, ncol(df[,-1]))
    outliers = subset(df, mahal > cutoff)    
    no_outliers = subset(df, mahal < cutoff)
    return(c(outliers, no_outliers))
}


remove_record = function(table, usernames){
    # remove all posts of users labeled as outliers
    for(username in usernames){
        del_query = paste("DELETE FROM ", table, " WHERE username='", username, "'", sep="")
        dbGetQuery(con, del_query)
    }
}

# collect user stats and then screen for outliers
consumers_stats = query_user_data("consumers_profile", "consumers_posts")
consumers_screening = outlier_screening(consumers_stats)    

# remove outliers from analysis (don't run this cell twice to avoid removing outliers after already excluding outliers)
# remove_record("consumers", consumers_screening[1])

<a id='propensity-score-matching'></a>
### G. Propensity Score Matching
To reduce bias of distribution overlap and different density weighting, we rebalance our data through matching non-treated users to treated ones on similar covariate values. First, we estimate a probit model of receiving treatment on the number of followers, the number of followings, the adoption speed of Instagram users, and the percentage of image posts relative to all types of media posts. Second, we compute the Mahalanobis distance for each treated and control user pair and select unique matches sequentially, in order of closeness of their Mahalanobis distances. We match without replacement such that control units are only allowed to be used as a match once. Each treatment unit is matched with a single control unit as a higher number of matches deteriorates matching quality significantly. Third, we conduct an imbalance check before and after matching of which the results are reported in the paper. 

In [3]:
# prepare data for propensity score matching (treatment/control country), follower/following count, image share, days since adoption
consumers_query = 
"
SELECT w.username, treatment_control, CAST(followers_count AS NUMERIC), CAST(following_count AS NUMERIC), image, days_since_adoption 
FROM consumers_profile w 
INNER JOIN 
    (SELECT username, CASE     
        WHEN country in ('australia', 'brazil', 'canada', 'italy') THEN 1   
        ELSE 0 END as treatment_control 
    FROM consumers_country cc WHERE country != 'brazil') x ON w.username = x.username

INNER JOIN (SELECT w.username, AVG(CAST(image_count AS DECIMAL) / CAST(posts_count AS DECIMAL)) as image 
            FROM consumers_profile w 
            INNER JOIN (SELECT username, SUM(CASE WHEN content_type = 'GraphImage' THEN 1 END) as image_count 
            FROM consumers_posts GROUP BY username) l ON l.username = w.username GROUP BY w.username) y ON y.username = w.username 

INNER JOIN (SELECT username, DATE_PART('day', '2020-06-01'::timestamp - MIN(timestamp)) as days_since_adoption 
            FROM consumers_posts GROUP BY username) z ON z.username = y.username
"

PSM = function(df){  
    # propensity score matching
    Tr = cbind(as.vector(df$treatment_control))
    X = as.matrix(df[,c('followers_count', 'following_count', 'days_since_adoption', 'image')])
    
    # replace NAs in the image column with zero
    X[is.na(X)] = 0 
    
    glm1 = glm(Tr ~ X, family=binomial)
    
    rr1 = Match(Tr=Tr, X=glm1$fitted, replace = FALSE, Weight=1, M=1)
    summary(rr1)
  
    # check balancing properties (results may deviate between bootstrap iterations)
    MatchBalance(Tr ~ X, match.out = rr1, nboots=10000)
  
    # store indices of matched users
    treatment = data.frame(df[rr1$index.treated,'username'], 'treatment')
    colnames(treatment) = c("username", "type")
    control = data.frame(df[rr1$index.control,'username'], 'control')
    colnames(control) = c("username", "type")
    return(rbind(treatment, control))
}

consumers_PSM_input = dbGetQuery(con, consumers_query)

# lines below are commented to ensure consistency with paper (PSM results may slightly deviate for each run)
# consumers_PSM = PSM(consumers_PSM_input)
# dbWriteTable(con, "consumers_psm", consumers_PSM, overwrite = TRUE, row.names = FALSE) 

<a id='differences-in-differences'></a>
### H. Difference in Differences
In line with our hypotheses, we examine posting frequency (H1), variety (H2), and like behavior (H3). To this end, we query the local data base and apply a difference in differences (DiD) approach to estimate the effect of hiding like counts on our matched sample of users. 

We compare the outcome measures of Instagram users in the treatment countries with those in control countries. As Canadians enter the treatment group prior to Australians, Brazilians, and Italians, we estimate a DiD where the time variable is relative to the intervention date.

$Y_{it} = \alpha_i + \gamma_t + I_{t} + \tau_{i} \cdot I_{t}  + \epsilon_{it}$

where $Y_it$ is the dependent variable for user i at time t, 
$\alpha_i$ is a user-level fixed effect, 
$\gamma_t$ is a trend variable, 
$\tau_{i}$ is 1 of if user i was assigned to the treatment group and 0 otherwise, 
$I_{t}$ is 1 if the intervention was implemented at time t and 0 otherwise, 
$\epsilon_{it}$ is the error term for user i at time t 

User level fixed effects control for time-invariant user characteristics. Intervention 1 and 2 take place in late April and mid-July, respectively. Given above equation, we are especially interested in the coefficient estimate and significance of the interaction between the treatment group and intervention variables as this indicates whether treatment units respond significantly different to the intervention than control units. To account for any serial correlation, we use robust standard errors clustered at the user level.

#### H.1 Posting frequency
We run a difference in differences model on the monthly number of Instagram posts and interpret the coefficients. Reported R-Square values are obtained by running a linear model with user fixed effects. The model coefficients relate to the regression output as follows:

| Coefficient | Regression Output | 
| :--- | :--- |
| $\gamma_t$ | `counter` |
| $I_{t}$ | `interventionTRUE` |
| $\tau_{i} \cdot I_{t}$ | `treatment:interventionTRUE`|

In [4]:
# for each user collect the number of posts, mean number of likes per post, and mean number of comments per post in each month
posts_likes_comments_query = 
"
SELECT c.username, followers_count, following_count, treatment, 
CASE WHEN country = 'canada' THEN date_part('year', age(month, '2019-05-01')) * 12 + date_part('month', age(month, '2019-05-01'))
WHEN country != 'canada' THEN date_part('year', age(month, '2019-08-01')) * 12 + date_part('month', age(month, '2019-08-01')) END as months_since_intervention,
posts, likes, comments
FROM
(SELECT c.username, country, followers_count, following_count,
 to_date(concat_ws('-', date_part('year', timestamp), date_part('month', timestamp), '1'), 'YYYY-MM-DD') as month, 
 COUNT(DISTINCT(shortcode)) as posts, 
 CASE WHEN country IN ('australia', 'canada', 'italy') THEN 1 ELSE 0 END as treatment,
 AVG(total_likes) as likes,
AVG(total_comments) as comments
FROM consumers_posts c
INNER JOIN consumers_country cc ON cc.username = c.username
INNER JOIN consumers_profile cp ON c.username = cp.username 
INNER JOIN consumers_psm cpsm ON cpsm.username = c.username
WHERE CASE WHEN country = 'canada' THEN DATE(timestamp) >= '2018-04-30' AND DATE(timestamp) <= '2020-04-30'
WHEN country != 'canada' THEN DATE(timestamp) >= '2018-07-17' AND DATE(timestamp) <= '2020-07-17' END
GROUP BY c.username, followers_count, following_count, treatment, cc.country, to_date(concat_ws('-', date_part('year', timestamp), date_part('month', timestamp), '1'), 'YYYY-MM-DD')) as c
"

posts = dbGetQuery(con, posts_likes_comments_query)
posts$after = posts$months_since_intervention >= 0 # create boolean that indicates whether the intervention was in place in a given month
posts$counter = posts$months_since_intervention # copy variable for the panel data analysis (see below)
    
fill_missing_months = function(df){
    # if users do not post in a given month we do not have any record of this. From this we can deduce that the number of posts in that month equals zero. This function searches for missing months and adds these records to the data frame.
    for(username in unique(df$username)){
        df_user = df[df$username == username,] 
        min_counter = min(df_user[, 'counter'])
        max_counter = max(df_user[, 'counter'])
        
        for(counter in min_counter:max_counter){
            if(!counter %in% df_user$counter){
                df[nrow(df) + 1, ] = c(username, df_user[1,'followers_count'], df_user[1,'following_count'], df_user[1,'treatment'], counter, 0, 0, 0, df_user[1,'after'], counter)
            }
        }
    }
    num_columns = c('followers_count', 'following_count', 'treatment', 'months_since_intervention', 'posts', 'likes', 'comments', 'counter')
    df[, num_columns] = sapply(df[, num_columns], as.numeric) 
    return(df)
}

posts = fill_missing_months(posts)

# log-transform posting frequency to account for skewness
posts[posts$posts == 0, 'posts'] = 0.0001
posts$log_posts = log(posts$posts)
posts$after = as.logical(posts$after)
posts$before = 1-posts$after

# run a Hausman test comparing random and fixed effects 
df.p = pdata.frame(posts, index=c('username', 'months_since_intervention'))
fixed_effects_posts = plm(as.formula(paste('log_posts', '~ treatment + after + treatment:after + counter + counter:treatment + after:treatment:counter')), data=df.p, model='within')
random_effects_posts = plm(as.formula(paste('log_posts', '~ treatment + after + treatment:after + counter + counter:treatment + after:treatment:counter')), data=df.p, model='random')
phtest(fixed_effects_posts, random_effects_posts) # choose for fixed effects (p < .001)
summary(fixed_effects_posts)


	Hausman Test

data:  as.formula(paste("log_posts", "~ treatment + after + treatment:after + counter + counter:treatment + after:treatment:counter"))
chisq = 171.64, df = 5, p-value < 2.2e-16
alternative hypothesis: one model is inconsistent


Oneway (individual) effect Within Model

Call:
plm(formula = as.formula(paste("log_posts", "~ treatment + after + treatment:after + counter + counter:treatment + after:treatment:counter")), 
    data = df.p, model = "within")

Unbalanced Panel: n = 238, T = 1-25, N = 5016

Residuals:
       Min.     1st Qu.      Median     3rd Qu.        Max. 
-9.93207332 -1.00494633  0.00010968  1.39768620  8.21545595 

Coefficients:
                             Estimate Std. Error  t-value  Pr(>|t|)    
afterTRUE                    4.649928   0.149663  31.0693 < 2.2e-16 ***
counter                     -0.304943   0.010933 -27.8912 < 2.2e-16 ***
treatment:afterTRUE         -0.804188   0.222859  -3.6085 0.0003111 ***
treatment:counter           -0.147786   0.016576  -8.9157 < 2.2e-16 ***
treatment:afterTRUE:counter  0.465593   0.025173  18.4956 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    39812
Residual Sum of Squares: 25925
R-Squared:      

#### H.2 Variety

In [5]:
# collect data for within-subject image similarity
within_subject_tags_query = 
"
SELECT ist.username, CASE WHEN cc.country IN ('australia', 'canada', 'italy') THEN 1 ELSE 0 END as treatment, 
CASE WHEN before_after = 'before' THEN 0 ELSE 1 END as after, image_similarity 
FROM image_similarity_within_tags ist 
INNER JOIN consumers_psm cp ON cp.username = ist.username 
INNER JOIN consumers_country cc ON cc.username = ist.username;
"
within_subjects_tags = dbGetQuery(con, within_subject_tags_query)
within_subjects_tags = within_subjects_tags[!duplicated(within_subjects_tags),] 
within_subjects_tags$after = factor(within_subjects_tags$after)
within_subjects_tags$treatment = factor(within_subjects_tags$treatment)

# image similarity did not differ between both treatment conditions nor before and after the intervention
ezANOVA(data = within_subjects_tags, 
        wid = username, 
        within = .(after), 
        between = .(treatment),
        dv = image_similarity)

Warning message:
“Converting "username" to factor for ANOVA.”Warning message:
“Data is unbalanced (unequal N per group). Make sure you specified a well-considered value for the type argument to ezANOVA().”

,Effect,DFn,DFd,F,p,p<.05,ges
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
2,treatment,1,233,0.009812402,0.9211777,,3.722437e-05
3,after,1,233,0.271603036,0.6027534,,1.352672e-04
4,treatment:after,1,233,0.179523063,0.6721743,,8.941245e-05


In [6]:
# collect data for between-subjects image similarity (only compare treatment units with treatment units or control units with control units)
between_subjects_tags_query = 
"
SELECT username1, username2, username1_2, 
CASE WHEN before_after = 'after' THEN 1 ELSE 0 END as after,
CASE WHEN cc1.country IN ('australia', 'canada', 'italy') THEN 1 ELSE 0 END as treatment,
CAST(similarity as numeric)
FROM image_similarity_between_tags i
INNER JOIN consumers_country cc1 ON i.username1 = cc1.username
INNER JOIN consumers_country cc2 ON i.username2 = cc2.username
WHERE (cc1.country IN ('australia', 'canada', 'italy') AND cc2.country IN ('australia', 'canada', 'italy'))
OR (cc1.country NOT IN ('australia', 'canada', 'italy') AND cc2.country NOT IN ('australia', 'canada', 'italy'))
"
between_subjects_tags = dbGetQuery(con, between_subjects_tags_query)

# remove duplicates
between_subjects_tags = between_subjects_tags[!duplicated(between_subjects_tags[,c('username1_2', 'after')]),]
between_subjects_tags$after = factor(between_subjects_tags$after)
between_subjects_tags$treatment = factor(between_subjects_tags$treatment)

# the treatment group reacted significantly different to the intervention than the control group (repeated-measures ANOVA)
ezANOVA(data = between_subjects_tags, 
        wid = username1_2, 
        within = .(after), 
        between = .(treatment),
        dv = similarity)

Warning message:
“Converting "username1_2" to factor for ANOVA.”Warning message:
“Data is unbalanced (unequal N per group). Make sure you specified a well-considered value for the type argument to ezANOVA().”

,Effect,DFn,DFd,F,p,p<.05,ges
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
2,treatment,1,13810,71.801250,2.618727e-17,*,4.617082e-03
3,after,1,13810,8.931177,2.808368e-03,*,6.974213e-05
4,treatment:after,1,13810,69.793206,7.208961e-17,*,5.447451e-04


#### H.3 Like Behavior

In [7]:
likes = dbGetQuery(con, posts_likes_comments_query)
likes$after = likes$months_since_intervention >= 0 
likes$counter = likes$months_since_intervention

for(counter in 1:nrow(likes)){
    if(likes[counter, 'likes'] == 0){
        likes[counter, 'log_likes'] = 0 # to avoid log-transformation issues
    } else {
        likes[counter, 'log_likes'] = log(likes[counter, 'likes'])  
    }         
}

df.p = pdata.frame(likes, index=c('username', 'months_since_intervention'))
df.p$log_likes1000 = df.p$log_likes * 1000 # so that coefficients can be easier interpreted
fixed_effects_likes = plm(as.formula(paste('log_likes1000', "~ treatment + after + treatment:after + counter + followers_count + following_count")), data=df.p, model='within')
random_effects_likes = plm(as.formula(paste('log_likes1000', "~ treatment + after + treatment:after + counter + followers_count + following_count")), data=df.p, model='random')
phtest(fixed_effects_likes, random_effects_likes) # choose for random effects (p > .05)

# number of likes goes down after the intervention, especially among small-scale audiences
summary(random_effects_likes)


	Hausman Test

data:  as.formula(paste("log_likes1000", "~ treatment + after + treatment:after + counter + followers_count + following_count"))
chisq = 4.7479, df = 3, p-value = 0.1912
alternative hypothesis: one model is inconsistent


Oneway (individual) effect Random Effect Model 
   (Swamy-Arora's transformation)

Call:
plm(formula = as.formula(paste("log_likes1000", "~ treatment + after + treatment:after + counter + followers_count + following_count")), 
    data = df.p, model = "random")

Unbalanced Panel: n = 238, T = 1-25, N = 4436

Effects:
                   var  std.dev share
idiosyncratic 111290.1    333.6 0.182
individual    498599.0    706.1 0.818
theta:
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.5728  0.8862  0.8974  0.8926  0.9040  0.9059 

Residuals:
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-2149.22  -180.64    19.13     0.36   190.44  1640.88 

Coefficients:
                       Estimate  Std. Error z-value  Pr(>|z|)    
(Intercept)         3280.592227   93.083612 35.2435 < 2.2e-16 ***
treatment            -94.735094   92.765368 -1.0212  0.307144    
afterTRUE            -66.739973   22.063356 -3.0249  0.002487 ** 
counter                6.150829    1.444412  4.2584 2.059e-05

### I. Randomized Experiment
<a id="experiment"></a>

<h5> I.1 Manipulation</h5>
<p>Our responses were recruited via a questionnaire-based <a href="https://tilburgss.co1.qualtrics.com/jfe/form/SV_brsLIHF0unNdsBT">experiment</a> on Prolific. We required workers to be aged between 18 to 30 years and active Instagram users to ensure they are familiar with the platform dynamics. Participants were randomly assigned to one of three same-gender conditions: high likes (128), low likes (15), and hidden likes. Individuals who self-identify as "Other" in the gender question were shown a picture of a woman.</p>

<img src="./images/experiment_pictures.png" align="left" width="800px"/>

<h5 style="clear: both;"> I.2 Target- and self-ratings</h5>
<p>They were asked to imagine that the person portrayed on the photo was their neighbor. Then, we asked participants to make specific evaluations of themselves and the target person in terms of likeability, popularity, and attractiveness:</p>

<img src="./images/target_self_ratings.png" alt="Questionnaire self and target ratings" align="left" width="600px"/>

<h5 style="clear: both;"> I.3 User behavior & demographics</h5>
<p>Thereafter, we measured user’s intention to like, comment, or share the picture after viewing the post. Finally, the questionnaire concluded by collecting the frequency of Instagram use and demographic information of the participants such as age and ethnicity. We received 600 responses which we analyze in the code blocks below.</p>

In [8]:
# import data
df = dbGetQuery(con, "SELECT * FROM experiment")

# exclude unknown genders (users who filled out "Other")
df = df[df$gender %in% c(1,2),]

# convert to factor
df$ethnicity = as.factor(df$ethnicity)

# construct reliability (Cronbach-Alpha)
cr_other_evaluation = alpha(df[,c('other.evaluation_1', 'other.evaluation_2', 'other.evaluation_3')]) #CR: 0.67
cr_self_evaluation = alpha(df[,c('self.evaluation_1', 'self.evaluation_2', 'self.evaluation_3')]) #CR: 0.78

# calculate aggregated target and self-ratings
df$target_rating = (df$other.evaluation_1 + df$other.evaluation_2 + df$other.evaluation_3)/3
df$self_rating = (df$self.evaluation_1 + df$self.evaluation_2 + df$self.evaluation_3)/3

# derive relative self-esteem (difference between target and self-ratings)
df$difference_evaluation = df$target_rating - df$self_rating

# there is a significant effect of the like count condition on relative self-esteem
anova = aov(difference_evaluation ~ condition + gender + age + ethnicity + instagram_usage, data=df)
print(summary(anova))

TukeyHSD(anova, which = "condition")

                 Df Sum Sq Mean Sq F value Pr(>F)  
condition         2   12.2   6.087   3.305 0.0374 *
gender            1    4.6   4.612   2.504 0.1141  
age               1    3.1   3.120   1.694 0.1936  
ethnicity         4   10.7   2.671   1.450 0.2161  
instagram_usage   1    4.9   4.913   2.667 0.1030  
Residuals       582 1072.0   1.842                 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Warning message in replications(paste("~", xx), data = mf):
“non-factors ignored: gender”Warning message in replications(paste("~", xx), data = mf):
“non-factors ignored: age”Warning message in replications(paste("~", xx), data = mf):
“non-factors ignored: instagram_usage”

  Tukey multiple comparisons of means
    95% family-wise confidence level

Fit: aov(formula = difference_evaluation ~ condition + gender + age + ethnicity + instagram_usage, data = df)

$condition
                   diff        lwr          upr     p adj
high-hidden  0.05117642 -0.2718321  0.374184944 0.9264655
low-hidden  -0.27268164 -0.5975855  0.052222200 0.1200701
low-high    -0.32385806 -0.6400208 -0.007695346 0.0432387


In [9]:
# here we conduct a similar analysis but this time as a 2 (source: self or target) X 3 (likes: low, high, hidden) mixed-model ANOVA (Vogel et al., 2014)
# this approach gives comparable results for the interaction between source and likes
df_melt = melt(df, colnames(df)[-c(17,18)])
anova_source = aov(value ~ variable + condition + variable:condition + gender + age + ethnicity + instagram_usage, data=df_melt)
summary(anova_source)

                     Df Sum Sq Mean Sq F value   Pr(>F)    
variable              1  372.4   372.4 356.032  < 2e-16 ***
condition             2    1.8     0.9   0.869  0.41977    
gender                1    0.0     0.0   0.022  0.88265    
age                   1    6.0     6.0   5.737  0.01677 *  
ethnicity             4   16.0     4.0   3.831  0.00424 ** 
instagram_usage       1   48.8    48.8  46.636 1.37e-11 ***
variable:condition    2    6.1     3.0   2.910  0.05486 .  
Residuals          1171 1224.8     1.0                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

<h5 style="clear: both;">Target and self-ratings (low, high, hidden like count)</h5>
<img src="./images/target_self_rating_chart.png" width="500px" align="left" />

<h5 style="clear: both;">Difference in target and self-ratings</h5>
<img src="./images/rating_difference.png" width="500px" align="left" />

In [10]:
# follow-up user behavior as a function of the like count
aggregate(df[,c('instagram_actions_1', 'instagram_actions_2', 'instagram_actions_3', 'instagram_actions_4')], list(df$condition), mean)

Group.1,instagram_actions_1,instagram_actions_2,instagram_actions_3,instagram_actions_4
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
hidden,3.362162,1.686486,2.713514,1.870270
high,3.368932,1.635922,2.509709,1.781553
low,3.368159,1.716418,2.761194,2.049751


In [11]:
# the manipulation (condition) did not affect any of the dependent measures
manova_results = manova(cbind(instagram_actions_1, instagram_actions_2, instagram_actions_4) ~ condition + gender + age + ethnicity + instagram_usage, data = df)
summary(manova_results)

                 Df   Pillai approx F num Df den Df    Pr(>F)    
condition         2 0.005771   0.5605      6   1162   0.76200    
gender            1 0.029619   5.9012      3    580   0.00057 ***
age               1 0.094243  20.1162      3    580 2.050e-12 ***
ethnicity         4 0.071790   3.5672     12   1746 2.840e-05 ***
instagram_usage   1 0.058748  12.0668      3    580 1.135e-07 ***
Residuals       582                                              
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [12]:
# ordered probit regression - posting frequency (example)
# note that these type of models are prefered given the ordinal scale of the answer response options (yet a simple lm-model gives comparable results)
df$instagram_actions_4_fact = as.factor(df$instagram_actions_4)
post_probit = polr(instagram_actions_4_fact ~ condition + gender + age + ethnicity + instagram_usage + self_rating + target_rating, data = df, Hess = TRUE, method='probit')
print(summary(post_probit))
print(pR2(post_probit))
ocME(post_probit)$out

Call:
polr(formula = instagram_actions_4_fact ~ condition + gender + 
    age + ethnicity + instagram_usage + self_rating + target_rating, 
    data = df, Hess = TRUE, method = "probit")

Coefficients:
                   Value Std. Error t value
conditionhigh   -0.08909    0.10786 -0.8260
conditionlow     0.07798    0.10807  0.7216
gender           0.08151    0.08741  0.9325
age              0.03278    0.01232  2.6613
ethnicity2       0.33974    0.13819  2.4585
ethnicity3       0.05818    0.15940  0.3650
ethnicity4      -0.11820    0.12859 -0.9192
ethnicity5      -0.07347    0.31183 -0.2356
instagram_usage  0.05618    0.03931  1.4290
self_rating      0.38661    0.04185  9.2378
target_rating   -0.01271    0.04856 -0.2617

Intercepts:
    Value   Std. Error t value
0|1  1.9932  0.4381     4.5492
1|2  2.7028  0.4407     6.1333
2|3  3.1568  0.4432     7.1225
3|4  3.7745  0.4491     8.4043
4|5  4.2121  0.4558     9.2417
5|6  4.8627  0.4695    10.3579

Residual Deviance: 1967.76 
AIC: 2001.7

,effect,error,t.value,p.value
conditionhigh,0.027,0.033,0.818,0.414
conditionlow,-0.024,0.032,-0.729,0.467
gender,-0.025,0.027,-0.932,0.352
age,-0.010,0.004,-2.656,0.008
ethnicity2,-0.093,0.034,-2.759,0.006
ethnicity3,-0.017,0.047,-0.372,0.710
ethnicity4,0.037,0.041,0.894,0.372
ethnicity5,0.023,0.100,0.230,0.818
instagram_usage,-0.017,0.012,-1.428,0.154
self_rating,-0.118,0.013,-8.888,0.000


<img src="./images/instagram_header.png" align="left"/>

*Klaasse Bos, R.J. (2020). Web Appendix: Goodbye Likes, Hello Mental Health: How Hiding Like Counts Affects User Behavior & Self-Esteem.*